# Tests per agegroup
Data extracted from weekly report of RIVM

In [38]:
import os
import pandas as pd
import tabula
import requests
from bs4 import BeautifulSoup
import PyPDF2
import re
from tqdm.notebook import tqdm
from datetime import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

rivm_url = 'https://www.rivm.nl/coronavirus-covid-19/actueel/wekelijkse-update-epidemiologische-situatie-covid-19-in-nederland'
response = requests.get(rivm_url)
rivm_pdf_data_loc = './pdfs/'

In [39]:
url = []
href = []
period = []
name = []
datetime = []
week = []
filename = []

def columns(l):
    def ll(l):
        return -l.count('')
    
    if 'http' in l:
        l = l[ll(l) + 20:]
    
    href.append(l)
    period.append(pd.Period(l[ll(l) + 22 : ll(l) + 29],"M"))
    name.append(l[ll(l) + 65 : -4])
    datetime.append(pd.to_datetime(l[ll(l) + 65 : ll(l) + 73], format="%Y%m%d"))
    filename.append(l[ll(l) + 65:])
    url.append('https://www.rivm.nl'+l)

In [40]:
soup = BeautifulSoup(response.text, 'html.parser') 
aa = soup.find_all('a')
for a in aa:
    link = a.get('href')
    if link == None:
        pass
    elif link.endswith('.pdf'):
        columns(link)

pdfs = pd.DataFrame([period, datetime, week, name, filename, href, url]).T
pdfs = pdfs.rename(columns={0:'period',1:'date',2:'week',3:'name',4:'filename',5:'href',6:'url'})
pdfs['week'] = pdfs.date.dt.isocalendar().week
pdfs.sort_values(by='period', inplace=True)

In [41]:
def extract_tabel(pdf,date):
    
    def check_tabel(value):
        for key in checks:
            if checks[key]:
                pass
            elif isinstance(value,str) and key in value:
                checks[key] = True
    
    def create_agegroup_tabel(tabel):
        #print(tabel)
        if checks['Leeftijdsgroep'] and checks['fgelopen']:
            category = 'Leeftijd'
        elif checks['Groep'] and checks['fgelopen'] and checks['Kinderen']:
            category = 'Groep'
        elif checks['Groep'] and checks['Kinderen']:
            category = 'Groep'
        else:
            category = ''
        #print('cat='+category)
        
        if category != '':
            #tabel.dropna(inplace=True)
            if tabel.empty:
                #print("df empty")
                pass
            else:
                #print('cat='+category)
                first_column = tabel.columns[0]
                #first_column = 'Unnamed: 0'
                if len(tabel.columns) > 1:
                    second_column = tabel.columns[1]
                agegroup_tabel = tabel.loc[(tabel[first_column].str.count('-') > 0)]                
                if category == 'Leeftijd':
                    agegroup_tabel = agegroup_tabel.append(tabel.loc[(tabel[first_column].str.find('+') > 0)])
                    agegroup_tabel.rename(columns={first_column:'Leeftijdsgroep',second_column:date}, inplace=True)
                elif category == 'Groep':
                    agegroup_tabel = agegroup_tabel.append(tabel.loc[(tabel[first_column].str.count('fgelopen') > 0)])
                    agegroup_tabel = agegroup_tabel.append(tabel.loc[(tabel[first_column].str.count('Groep') > 0)])
                agegroup_tabel = agegroup_tabel.sort_index()
                agegroup_tabel['Date'] = date
                csv = pdf[:-4]+'-'+category+'.csv'
                agegroup_tabel.to_csv(csv, index=False)
                #print('CSV CREATED')
                #print(agegroup_tabel)
                return True
        #else:
            #print("NOTHING")
    
    #print('extracting '+pdf)
    #print(pdf)
    found_pages = []
    object = PyPDF2.PdfFileReader(pdf)
    NumPages = object.getNumPages()
    String = "SARS-COV-2.*TESTEN.*AFGENOMEN.*DOOR.*DE.*GGD"
    
    for i in tqdm(range(0, NumPages)):
        PageObj = object.getPage(i)
        Text = PageObj.extractText()
        if re.search(String,Text):
            found_pages.append(i)
    
    if len(found_pages) == 0:
        print("NO CHAPTER FOUND in "+str(pdf))
    else:
        tabels = tabula.read_pdf(input_path=pdf, pages=found_pages, guess=True, stream=False, multiple_tables=True)
        csv_created = None
        for tabel in tabels:
            checks = {
                'Leeftijdsgroep': False,
                'fgelopen': False,
                'Groep': False,
                'Kinderen': False}
            for column in tabel:
                check_tabel(column)
                for i in tabel[column].values:
                    check_tabel(i)
            #print(checks)
            create_agegroup_tabel(tabel)

In [42]:
# download all pdfs
print('Checking for new weekreport')
for row in pdfs.itertuples():
    pdf = rivm_pdf_data_loc+row.filename
    try:
        try:
            csv = pdf[:-4]+'-Leeftijd.csv'
            os.stat(csv)
        except:
            csv = pdf[:-4]+'-Groep.csv'
            os.stat(csv)
        #print('skiping '+csv+' exists')
    except:
        #print(pdf)
        try:
            os.stat(pdf)
        except:
            #print('downloading '+row.filename)
            r = requests.get(row.url, stream=True)
            with open(pdf, 'wb') as f:
                f.write(r.content)
        finally:
            #print("extracting tabels from pdf")
            extract_tabel(pdf, row.date)
print("DONE")

Checking for new weekreport


  0%|          | 0/42 [00:00<?, ?it/s]

DONE


In [43]:
#debug = 0
#untill = 42

def add_data_from(f):    
    def merge_split(df):
        df_temp = df[df.columns].apply(
            lambda x: ' '.join(x.dropna().astype(str)),
            axis=1
        )
        df_temp = pd.DataFrame(df_temp.str.split(expand=True))
        return df_temp
    
    #if debug == untill:
    #    print(filename)
    #    print(debug)
    df = pd.read_csv(rivm_pdf_data_loc+f)
    # selecting usable rows
    if df.iloc[0,0] == 'Groep Aantal positief Aantal getest Percentage positief':
        text_in_column = 'Tabel 12: Aantal testen en percentage positief per doelgroep in de afgelopen twee weken1.'
        if (text_in_column in df.iloc[:,0].values) == False:
            text_in_column = 'Tabel 13: Aantal testen en percentage positief per doelgroep in de afgelopen week1.'
        df = df.iloc[df.index[df.index[df.iloc[:,0] == text_in_column].values[0]:,...]]
    
    if df.columns[0] != 'Leeftijdsgroep':
        df = df[df.iloc[:,0].str.contains('jaar')]
    df.iloc[:,0].replace({'jaar2':'jaar', '70\+':'70 en ouder', '80\+':'80 en ouder'}, regex=True, inplace=True)
        
    # setting Date to index
    date = [int(i) for i in df.iloc[0,-1].split('-')]
    date = dt(date[0], date[1], date[2])
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    df.index = pd.DatetimeIndex(df.index)

    # setting Agegroup
    df_temp = merge_split(df)
    if '-' in df_temp.iloc[0,0]:
        df_temp2 = df_temp.iloc[:,0].str.split('-',expand=True)
        df_temp2['Agegroup'] = df_temp2.iloc[:,0]+' - '+df_temp2.iloc[:,1]
        if '95+' in df_temp2.iloc[-1,0]:
            df_temp2.iloc[-1,-1] = '95 en ouder'
        df_temp.iloc[:,0] = df_temp2['Agegroup']
        df_temp = merge_split(df_temp)
    
    df['Agegroup'] = df_temp.iloc[:,0]+' '+df_temp.iloc[:,1]+' '+df_temp.iloc[:,2]
    # setting values
    if date > dt(2020, 12, 1):
        df['Tested positive'] = df_temp.iloc[:,-2]
        df['Total tested'] = df_temp.iloc[:,-3]
    else:
        df['Tested positive'] = df_temp.iloc[:,-3]
        df['Total tested'] = df_temp.iloc[:,-2]
    df['cat'] = f.find('Leeftijd')
    df['cat'].replace({-1: 'Groep', 14: 'Leeftijd', 20: 'Leeftijd', 22: 'Leeftijd'}, inplace=True)

    #if debug == untill:
    #    print(df_temp)
    df = df[['Agegroup', 'Tested positive', 'Total tested', 'cat']]
    return df

In [44]:
# clean up extracted tabels
data = pd.DataFrame()
csvs = os.listdir(rivm_pdf_data_loc)
csvs.sort()
first = True
for filename in csvs:
    skip = False
    #if debug == untill+1:
    #    break
    if '.csv' in filename:
        if filename in ['20210413_1259-Groep.csv']:
            #debug += 1
            pass
        else:
            data = pd.concat([data,add_data_from(filename)])
            #debug += 1

data.index = pd.DatetimeIndex(data.index)
data['Tested positive'] = data['Tested positive'].astype(int)
data['Total tested'] = data['Total tested'].astype(int)

data_temp = data['cat'].reset_index().drop_duplicates().groupby('Date').count()
data_temp.rename(columns={'cat':'count'}, inplace=True)
data = pd.merge(data,data_temp, on='Date')
data['cat_count'] = data['cat']+data['count'].astype(str)
data = data[data['cat_count'] != 'Groep2']
data.drop(columns=['cat', 'count', 'cat_count'], inplace=True)

#calculate percentage
data['Percentage'] = data['Tested positive'] / data['Total tested'] * 100
data

,Agegroup,Tested positive,Total tested,Percentage
Date,,,,
2020-07-07,Kinderen 0-6 jaar,25,7033,0.355467
2020-07-07,Kinderen 7-12 jaar,50,19446,0.257122
2020-07-07,Kinderen 13-18 jaar,97,8661,1.119963
2020-07-21,Kinderen 0-6 jaar,5,898,0.556793
2020-07-21,Kinderen 7-12 jaar,23,6221,0.369715
...,...,...,...,...
2021-04-13,40 - 49,6972,68309,10.206561
2021-04-13,50 - 59,7558,60096,12.576544
2021-04-13,60 - 69,4410,42946,10.268710


In [45]:
data['Agegroup'].replace({
    'Kinderen 0-3 jaar': '0 - 3', 
    'Kinderen 0-6 jaar': '0 - 6', 
    'Kinderen 4-11 jaar': '4 - 11', 
    'Kinderen 4-11 jaar3': '4 - 11', 
    'Kinderen 4-12 jaar': '4 - 12', 
    'Kinderen 7-12 jaar': '7 - 12', 
    'Kinderen 12-17 jaar': '12 - 17',
    'Kinderen 12-17 jaar4': '12 - 17',
    'Kinderen 13-17 jaar': '13 - 17',
    'Kinderen 13-18 jaar': '13 - 18', 
    'Jongvolwassenen 18-24 jaar': '18 - 24'}, inplace=True)

In [46]:
data['Group'] = data['Agegroup'].copy()
data.loc[:,'Group'].replace({
    '0 - 3': 'Kinderen', 
    '0 - 4': 'Kinderen',
    '0 - 6': 'Kinderen', 
    '0 - 12': 'Basisschool',
    '5 - 9': 'Basisschool',
    '10 - 14': 'Basisschool',
    '4 - 12': 'Basisschool', 
    '7 - 12': 'Basisschool', 
    '12 - 17': 'Pubers',
    '13 - 17': 'Pubers',
    '13 - 18': 'Pubers', 
    '15 - 19': 'Pubers', 
    '18 - 24': 'Jongvolwassenen',
    '20 - 24': 'Jongvolwassenen', 
    '25 - 29': 'Jongvolwassenen',
    '30 - 34': 'Volwassenen',
    '30 - 39': 'Volwassenen', 
    '35 - 39': 'Volwassenen',
    '40 - 44': 'Volwassenen',
    '40 - 49': 'Volwassenen',
    '45 - 49': 'Volwassenen',
    '50 - 54': 'Volwassenen',
    '50 - 59': 'Volwassenen',
    '55 - 59': 'Volwassenen',
    '60 - 64': 'Volwassenen',
    '60 - 69': 'Volwassenen', 
    '65 - 69': 'Volwassenen',
    '70 en ouder': '70 en ouder',
    '70 - 74': '70 en ouder',
    '70 - 79': '70 en ouder', 
    '75 - 79': '70 en ouder',
    '80 en ouder': '70 en ouder',
    '80 - 84': '70 en ouder',
    '85 - 89': '70 en ouder',
    '90 - 94': '70 en ouder'}, inplace=True)

In [47]:
data['Agerange'] = data['Agegroup'].copy()
data.loc[:,'Agerange'].replace({
    '0 - 3': '0 - 12', 
    '0 - 4': '0 - 12',
    '0 - 6': '0 - 12', 
    '0 - 12': '0 - 12',
    '5 - 9': '0 - 12',
    '10 - 14': 'groep 7/8 + brugklas',
    '4 - 12': '0 - 12', 
    '7 - 12': '0 - 12', 
    '12 - 17': '13 - 24',
    '13 - 17': '13 - 24',
    '13 - 18': '13 - 24', 
    '15 - 19': '13 - 24', 
    '18 - 24': '13 - 24',
    '20 - 24': '13 - 24', 
    '25 - 29': '25 - 39',
    '30 - 34': '25 - 39',
    '30 - 39': '25 - 39', 
    '35 - 39': '25 - 39',
    '40 - 44': '40 - 49',
    '40 - 49': '40 - 49',
    '45 - 49': '40 - 49',
    '50 - 54': '50 - 59',
    '50 - 59': '50 - 59',
    '55 - 59': '50 - 59',
    '60 - 64': '60 - 69',
    '60 - 69': '60 - 69', 
    '65 - 69': '60 - 69',
    '70 en ouder': '70 en ouder',
    '70 - 74': '70 en ouder',
    '70 - 79': '70 en ouder', 
    '75 - 79': '70 en ouder',
    '80 en ouder': '70 en ouder',
    '80 - 84': '70 en ouder',
    '85 - 89': '70 en ouder',
    '90 - 94': '70 en ouder',
    '95 en ouder': '70 en ouder'}, inplace=True)

In [48]:
%matplotlib widget
import seaborn as sns # Calls in seaborn

# These lines generate the data to be plotted
kinderen = data[data.loc[:,'Agerange'] == '0 - 12']
kindere = kinderen.groupby(by=kinderen.index).mean()
x = kinderen.index
y = kinderen['Total tested']
y1 = kinderen['Tested positive']
y2 = kinderen['Percentage']

fig, ax1 = plt.subplots() # initializes figure and plots

ax2 = ax1.twinx() # applies twinx to ax2, which is the second y axis. 

sns.lineplot(x = x, y = y, ax = ax1, estimator='max') # plots the first set of data, and sets it to ax1. 
#sns.lineplot(x = x, y = y, ax = ax1, estimator=None, style=True, dashes=[(2, 2)], color='lightblue') # plots the first set of data, and sets it to ax1. 
sns.lineplot(x = x, y = y1, ax = ax1, estimator='max') # plots the first set of data, and sets it to ax1. 
sns.lineplot(x = x, y = y2, ax = ax2, color='mediumseagreen', estimator='max') # plots the first set of data, and sets it to ax1. 

# these lines add the annotations for the plot. 
ax1.set_xlabel('Date')
ax1.set_ylabel('Counts')
ax2.set_ylabel('Percentage', color='mediumseagreen')

ax1.set_title(label='0 - 12 jaar')
fig.legend(labels=['Total tested', 'Tested positive', 'Percentage'], loc='upper left')
plt.show(); # shows the plot.

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
%matplotlib widget
sns.lineplot(data=data, x="Date", y="Total tested", hue="Agerange", estimator='max').set_title('Aantal testen')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Aantal testen')

In [50]:
%matplotlib widget
sns.lineplot(data=data, x="Date", y="Tested positive", hue="Agerange", estimator='max').set_title('Aantal positieve testen')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Aantal positieve testen')

In [51]:
%matplotlib widget
sns.lineplot(data=data, x="Date", y="Total tested", hue="Group", estimator='max').set_title('Aantal testen')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Aantal testen')

In [52]:
%matplotlib widget
sns.lineplot(data=data, x="Date", y="Tested positive", hue="Group", estimator='max').set_title('Aantal positieve testen')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Aantal positieve testen')

In [53]:
%matplotlib widget
import seaborn as sns # Calls in seaborn

# These lines generate the data to be plotted
grouped = data[(data['Group'] == 'Kinderen') | (data['Group'] == 'Basisschool') | (data['Group'] == 'Pubers') | (data['Group'] == '4 - 11')].groupby(by=['Date','Group']).sum()
grouped.reset_index(level='Group', inplace=True)
grouped['Percantage'] = grouped['Tested positive'] / grouped['Total tested'] * 100

x = grouped.index
y = grouped['Total tested']
y1 = grouped['Tested positive']
y2 = grouped['Percentage']

fig, ax1 = plt.subplots() # initializes figure and plots

ax2 = ax1.twinx() # applies twinx to ax2, which is the second y axis. 

sns.lineplot(x = x, y = y, ax = ax1, estimator='max') # plots the first set of data, and sets it to ax1. 
#sns.lineplot(x = x, y = y, ax = ax1, estimator=None, style=True, dashes=[(2, 2)], color='lightblue') # plots the first set of data, and sets it to ax1. 
sns.lineplot(x = x, y = y1, ax = ax1, estimator='max') # plots the first set of data, and sets it to ax1. 
sns.lineplot(x = x, y = y2, ax = ax2, color='mediumseagreen', estimator='max') # plots the first set of data, and sets it to ax1. 

# these lines add the annotations for the plot. 
ax1.set_xlabel('Date')
ax1.set_ylabel('Counts')
ax2.set_ylabel('Percentage', color='mediumseagreen')

ax1.set_title(label='Data in groups for children')
fig.legend(labels=['Total tested', 'Tested positive', 'Percentage'], loc='upper left')
plt.show(); # shows the plot.

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## What can be used from the reports

### Chapter 4 agegroups
Reported, Hospitalised, Deceased agegroups

cumulatief